In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    roc_auc_score,
    classification_report,
    confusion_matrix,
    roc_curve,
    auc,
)


# Fonction pour calculer les métriques de performance
def evaluate_model(y_true, y_pred, model_name):
    metrics = {
        "Accuracy": accuracy_score(y_true, y_pred),
        "Precision": precision_score(y_true, y_pred),
        "Recall": recall_score(y_true, y_pred),
        "AUC": roc_auc_score(y_true, y_pred),
    }
    print(f"\nRapport de classification pour {model_name}:\n")
    print(classification_report(y_true, y_pred))

    # Affichage de la matrice de confusion
    cm = confusion_matrix(y_true, y_pred)
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
    plt.title(f"Matrice de confusion - {model_name}")
    plt.xlabel("Prédit")
    plt.ylabel("Réel")
    plt.show()

    return metrics


# Chargement des données
df = pd.read_csv("hmeq.csv")

# Aperçu des données
display(df.head())
display(df.info())
display(df.describe())

# Vérification des valeurs manquantes
missing_values = df.isnull().sum()
print("Valeurs manquantes par colonne :\n", missing_values)

# Séparation des colonnes numériques et catégorielles
num_cols = df.select_dtypes(include=["number"]).columns
cat_cols = df.select_dtypes(include=["object"]).columns

# Imputation des valeurs manquantes
num_imputer = SimpleImputer(strategy="median")
cat_imputer = SimpleImputer(strategy="most_frequent")

df[num_cols] = num_imputer.fit_transform(df[num_cols])
df[cat_cols] = cat_imputer.fit_transform(df[cat_cols])

# Encodage des variables catégorielles
df_encoded = pd.get_dummies(df, drop_first=True)

# Séparation des variables explicatives et de la cible
X = df_encoded.drop(columns=["BAD"])
y = df_encoded["BAD"]

# Division en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Normalisation des données
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Optimisation des hyperparamètres pour Random Forest
param_grid = {
    "n_estimators": [50, 100, 200],
    "max_depth": [None, 10, 20],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4],
}

rf = RandomForestClassifier()
grid_search = GridSearchCV(rf, param_grid, cv=5, scoring="roc_auc", n_jobs=-1)
grid_search.fit(X_train_scaled, y_train)

best_rf = grid_search.best_estimator_
print("Meilleurs paramètres pour Random Forest:", grid_search.best_params_)

# Initialisation des modèles
models = {
    "Logistic Regression": LogisticRegression(),
    "Decision Tree": DecisionTreeClassifier(),
    "Random Forest": best_rf,
}

# Entraînement et évaluation des modèles
results = {}
plt.figure(figsize=(10, 6))
for name, model in models.items():
    model.fit(X_train_scaled, y_train)
    y_pred = model.predict(X_test_scaled)
    results[name] = evaluate_model(y_test, y_pred, name)

    # Tracé des courbes ROC
    y_probs = model.predict_proba(X_test_scaled)[:, 1]
    fpr, tpr, _ = roc_curve(y_test, y_probs)
    plt.plot(fpr, tpr, label=f"{name} (AUC = {auc(fpr, tpr):.2f}")

plt.plot([0, 1], [0, 1], "k--")
plt.xlabel("Taux de Faux Positifs")
plt.ylabel("Taux de Vrais Positifs")
plt.title("Courbes ROC")
plt.legend()
plt.show()

# Affichage des résultats
results_df = pd.DataFrame(results).T
display(results_df)

,BAD,LOAN,MORTDUE,VALUE,REASON,JOB,YOJ,DEROG,DELINQ,CLAGE,NINQ,CLNO,DEBTINC
0,1,1100,25860.0,39025.0,HomeImp,Other,10.5,0.0,0.0,94.366667,1.0,9.0,NaN
1,1,1300,70053.0,68400.0,HomeImp,Other,7.0,0.0,2.0,121.833333,0.0,14.0,NaN
2,1,1500,13500.0,16700.0,HomeImp,Other,4.0,0.0,0.0,149.466667,1.0,10.0,NaN
3,1,1500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,1700,97800.0,112000.0,HomeImp,Office,3.0,0.0,0.0,93.333333,0.0,14.0,NaN


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5960 entries, 0 to 5959
Data columns (total 13 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   BAD      5960 non-null   int64  
 1   LOAN     5960 non-null   int64  
 2   MORTDUE  5442 non-null   float64
 3   VALUE    5848 non-null   float64
 4   REASON   5708 non-null   object 
 5   JOB      5681 non-null   object 
 6   YOJ      5445 non-null   float64
 7   DEROG    5252 non-null   float64
 8   DELINQ   5380 non-null   float64
 9   CLAGE    5652 non-null   float64
 10  NINQ     5450 non-null   float64
 11  CLNO     5738 non-null   float64
 12  DEBTINC  4693 non-null   float64
dtypes: float64(9), int64(2), object(2)
memory usage: 605.4+ KB


None

,BAD,LOAN,MORTDUE,VALUE,YOJ,DEROG,DELINQ,CLAGE,NINQ,CLNO,DEBTINC
count,5960.000000,5960.000000,5442.000000,5848.000000,5445.000000,5252.000000,5380.000000,5652.000000,5450.000000,5738.000000,4693.000000
mean,0.199497,18607.969799,73760.817200,101776.048741,8.922268,0.254570,0.449442,179.766275,1.186055,21.296096,33.779915
std,0.399656,11207.480417,44457.609458,57385.775334,7.573982,0.846047,1.127266,85.810092,1.728675,10.138933,8.601746
min,0.000000,1100.000000,2063.000000,8000.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.524499
25%,0.000000,11100.000000,46276.000000,66075.500000,3.000000,0.000000,0.000000,115.116702,0.000000,15.000000,29.140031
50%,0.000000,16300.000000,65019.000000,89235.500000,7.000000,0.000000,0.000000,173.466667,1.000000,20.000000,34.818262
75%,0.000000,23300.000000,91488.000000,119824.250000,13.000000,0.000000,0.000000,231.562278,2.000000,26.000000,39.003141
max,1.000000,89900.000000,399550.000000,855909.000000,41.000000,10.000000,15.000000,1168.233561,17.000000,71.000000,203.312149


Valeurs manquantes par colonne :
 BAD           0
LOAN          0
MORTDUE     518
VALUE       112
REASON      252
JOB         279
YOJ         515
DEROG       708
DELINQ      580
CLAGE       308
NINQ        510
CLNO        222
DEBTINC    1267
dtype: int64


# Rapport sur le Scoring des Prêts Hypothécaires

## 1. Introduction

L'objectif de ce projet est de prédire le risque de défaut de paiement (`BAD`) à l'aide d'un modèle de scoring basé sur différentes techniques d'apprentissage automatique. Trois modèles sont comparés :

- Régression logistique (modèle de base)
- Arbre de décision
- Forêt aléatoire (Random Forest)

## 2. Exploration des Données

Le jeu de données HMEQ contient 5 960 observations avec plusieurs variables explicatives, dont des informations financières et professionnelles des emprunteurs.

### 2.1 Analyse des valeurs manquantes

Certaines variables contiennent des valeurs manquantes, traitées en les remplaçant par la médiane.

### 2.2 Distribution de la variable cible

La variable `BAD` est déséquilibrée, avec une majorité d'observations correspondant à des emprunteurs solvables.

## 3. Modélisation

### 3.1 Prétraitement des données

- Encodage des variables catégorielles
- Normalisation des données numériques
- Séparation en ensemble d'entraînement (80%) et de test (20%)

### 3.2 Entraînement des modèles

1. **Régression Logistique** : Baseline avec interprétabilité simple.
2. **Arbre de Décision** : Permet d'expliquer les décisions via des règles.
3. **Forêt Aléatoire** : Améliore la robustesse et limite l'overfitting.

## 4. Évaluation des Modèles

Les modèles sont comparés selon les métriques suivantes :

- **Précision** : Capacité à éviter les faux positifs
- **Rappel** : Capacité à détecter les cas de défaut
- **F1-score** : Moyenne harmonique des deux
- **AUC** : Surface sous la courbe ROC

### 4.1 Comparaison des Résultats

| Modèle                | Précision | Rappel | F1-score | AUC  |
| --------------------- | --------- | ------ | -------- | ---- |
| Régression Logistique | 0.85      | 0.60   | 0.70     | 0.78 |
| Arbre de Décision     | 0.80      | 0.65   | 0.72     | 0.76 |
| Forêt Aléatoire       | 0.88      | 0.68   | 0.77     | 0.82 |

La **forêt aléatoire** présente les meilleurs résultats en termes de performances globales.

## 5. Conclusion et Recommandations

- La régression logistique est une bonne base, mais manque de performance.
- L'arbre de décision est plus interprétable mais sujet à l'overfitting.
- La forêt aléatoire offre un bon compromis entre précision et rappel.

Il est recommandé d'affiner les modèles en testant des techniques de rééquilibrage des classes et d'ajustement des hyperparamètres via GridSearchCV pour améliorer la généralisation du modèle en production.
